<a href="https://colab.research.google.com/github/redhood18/diabetes_predictor/blob/main/diabetes_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikeras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as p
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#data=p.DataFrame(temp_data,columns=['Unnamed:0','Age','Gender','BMI','Chol','TG','HDL','LDL','Cr','BUN','Diagnosis'])
#data.head()


In [ ]:
#shuffled_data=data.sample(frac=1)
#shuffled_data[:20]

In [ ]:
#shuffled_data=shuffled_data.drop("Unnamed:0",axis=1)
#shuffled_data.head()

In [ ]:
#shuffled_data.to_csv("drive/MyDrive/diabetes/shuffled_data.csv")
#new_data=p.read_csv("drive/MyDrive/diabetes/shuffled_data.csv",index_col=0)
#new_data.head()

In [ ]:
#new_data[:10]

In [ ]:
def plot_graph(data):
  column_name=list(data.columns.values)
  column_name=column_name[:9]
  for i in (column_name):
    plt.scatter(new_data[i],new_data['Diagnosis'])
    plt.xlabel(i)
    plt.ylabel("Diagnosis")
    plt.show()

#plot_graph(new_data)

In [ ]:
#testing_dataset=new_data.sample(frac=0.2,random_state=200)
#testing_dataset.head()
#training_dataset=new_data.drop(testing_dataset.index)
#training_dataset.head()
#print("Size of testing dataset:",testing_dataset.size)
#print("Size of training dataset:",training_dataset.size)
#print("Size of original dataset:",new_data.size)


In [ ]:
#testing_dataset.head()

In [ ]:
#training_dataset.head()

In [ ]:
#training_dataset.to_csv("drive/MyDrive/diabetes/train.csv")
#testing_dataset.to_csv("drive/MyDrive/diabetes/test.csv")

In [ ]:
train_data=p.read_csv("drive/MyDrive/diabetes/train.csv",index_col=0)
train_data.head()

,Age,Gender,BMI,Chol,TG,HDL,LDL,Cr,BUN,Diagnosis
593,51,M,27,6.00,1.10,1.40,1.30,41.0,3.30,1
3942,66,F,30,4.14,1.14,1.71,2.19,48.6,6.43,1
1560,37,F,23,4.07,1.19,1.09,2.33,43.1,6.10,0
881,41,M,24,4.96,3.65,1.34,2.66,73.4,3.79,0
1461,37,F,22,5.26,0.57,2.21,3.04,56.8,4.86,0


In [ ]:
train_data.size

41060

In [ ]:
train_data["Diagnosis"].value_counts()

0    2514
1    1592
Name: Diagnosis, dtype: int64

In [ ]:
train_data.isnull().sum()

Age          0
Gender       0
BMI          0
Chol         0
TG           0
HDL          0
LDL          0
Cr           0
BUN          0
Diagnosis    0
dtype: int64

In [ ]:
def converting_gender(train_data):
  train_data.loc[train_data["Gender"]=="M","Gender"]=1
  train_data.loc[train_data["Gender"]=="F","Gender"]=0
  train_data.loc[train_data["Gender"]=="m","Gender"]=1
  train_data.loc[train_data["Gender"]=="f","Gender"]=0
  return train_data

In [ ]:
input_data=train_data.drop("Diagnosis",axis=1)
x=input_data.copy()
y=train_data["Diagnosis"]
x=converting_gender(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.30, random_state=5)

In [ ]:
x_train_normalized = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
x_val_normalized = (x_validation - x_train.mean(axis=0)) / x_train.std(axis=0)

In [ ]:
tf.experimental.numpy.experimental_enable_numpy_behavior()
x_train=tf.convert_to_tensor(x_train.astype(np.float32))
x_validation=tf.convert_to_tensor(x_validation.astype(np.float32))
y_train=tf.convert_to_tensor(y_train.astype(np.float32))
y_validation=tf.convert_to_tensor(y_validation.astype(np.float32))

In [ ]:
x_train_normalized=tf.convert_to_tensor(x_train_normalized.astype(np.float32))
x_val_normalized=tf.convert_to_tensor(x_val_normalized.astype(np.float32))
y_train=tf.convert_to_tensor(y_train.astype(np.float32))
y_validation=tf.convert_to_tensor(y_validation.astype(np.float32))


(<tf.Tensor: shape=(4106, 9), dtype=float32, numpy=
 array([[ 51.      ,   1.      ,  27.      , ...,   1.3     ,  41.      ,
           3.3     ],
        [ 66.      ,   0.      ,  30.      , ...,   2.19    ,  48.6     ,
           6.43    ],
        [ 37.      ,   0.      ,  23.      , ...,   2.33    ,  43.1     ,
           6.1     ],
        ...,
        [ 63.      ,   1.      ,  23.      , ...,   3.09    ,  85.8     ,
           3.56    ],
        [ 61.      ,   1.      ,  22.      , ...,   3.05    , 105.9     ,
           5.85    ],
        [ 48.      ,   1.      ,  25.      , ...,   4.860753,  66.      ,
           5.16    ]], dtype=float32)>,
 <tf.Tensor: shape=(4106,), dtype=float32, numpy=array([1., 1., 0., ..., 1., 1., 1.], dtype=float32)>)

In [ ]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [ ]:
np.random.seed(42)
model=tf.keras.Sequential([tf.keras.layers.Dense(10,activation="relu"),tf.keras.layers.Dense(32,activation="relu"),tf.keras.layers.Dense(26,activation="relu"),tf.keras.layers.Dense(10,activation="relu"),tf.keras.layers.Dense(1,activation="sigmoid")])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=["accuracy"])
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="accuracy",patience=10)
model.fit(x_train, y_train, epochs=100, batch_size=32,validation_split=0.3,validation_data=(x_validation, y_validation))

param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.2, 0.3, 0.4],
    'neurons': [32, 64, 128],
    'epochs': [50, 100],
    'batch_size': [16, 32]
}

Epoch 1/100
90/90 [==============================] - 4s 10ms/step - loss: 0.7216 - accuracy: 0.6889 - val_loss: 0.5621 - val_accuracy: 0.7110
Epoch 2/100
90/90 [==============================] - 1s 8ms/step - loss: 0.5602 - accuracy: 0.7237 - val_loss: 0.5384 - val_accuracy: 0.7208
Epoch 3/100
90/90 [==============================] - 0s 5ms/step - loss: 0.5226 - accuracy: 0.7359 - val_loss: 0.5321 - val_accuracy: 0.7248
Epoch 4/100
90/90 [==============================] - 0s 5ms/step - loss: 0.5121 - accuracy: 0.7394 - val_loss: 0.5485 - val_accuracy: 0.7062
Epoch 5/100
90/90 [==============================] - 1s 6ms/step - loss: 0.5141 - accuracy: 0.7429 - val_loss: 0.5437 - val_accuracy: 0.7248
Epoch 6/100
90/90 [==============================] - 1s 6ms/step - loss: 0.5019 - accuracy: 0.7512 - val_loss: 0.5090 - val_accuracy: 0.7573
Epoch 7/100
90/90 [==============================] - 0s 5ms/step - loss: 0.4926 - accuracy: 0.7596 - val_loss: 0.5123 - val_accuracy: 0.7646
Epoch 8/100


In [ ]:
import os
import datetime
modeldir=os.path.join("drive/MyDrive/diabetes/models",datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
model_path=modeldir+"-"+".h5"
model.save(model_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def load_model(model_path):
  model=tf.keras.models.load_model(model_path,custom_objects={"KerasLayer":hub.KerasLayer})
  return model
loaded_model=load_model("drive/MyDrive/diabetes/models/20240405-19521712346745-.h5")

In [ ]:
test_data=p.read_csv("drive/MyDrive/diabetes/test.csv",index_col=0)

In [ ]:
test_data.head()

,Age,Gender,BMI,Chol,TG,HDL,LDL,Cr,BUN,Diagnosis
3151,33,M,26,4.70,1.96,1.410000,2.120000,71.000000,3.60,0
248,55,M,28,4.90,2.10,1.100000,2.500000,56.000000,5.40,1
2122,41,F,18,4.52,0.53,2.080000,2.400000,45.600000,3.77,0
5086,66,F,21,4.51,0.65,4.860753,4.860753,4.860753,4.87,1
3432,76,M,27,5.80,1.50,1.330000,3.270000,69.500000,5.90,0


In [ ]:
data=test_data.drop("Diagnosis",axis=1)
actual_diag=test_data["Diagnosis"]

In [ ]:
data=converting_gender(data)
data.head()

,Age,Gender,BMI,Chol,TG,HDL,LDL,Cr,BUN
3151,33,1,26,4.70,1.96,1.410000,2.120000,71.000000,3.60
248,55,1,28,4.90,2.10,1.100000,2.500000,56.000000,5.40
2122,41,0,18,4.52,0.53,2.080000,2.400000,45.600000,3.77
5086,66,0,21,4.51,0.65,4.860753,4.860753,4.860753,4.87
3432,76,1,27,5.80,1.50,1.330000,3.270000,69.500000,5.90


In [ ]:
x=tf.convert_to_tensor(data.astype(np.float32))

In [ ]:
test_predictions=loaded_model.predict(x,verbose=1)
binary_predictions=(test_predictions>=0.5).astype(int)
binary_predictions

33/33 [==============================] - 0s 2ms/step


array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [ ]:
from sklearn.metrics import accuracy_score
actual_diag=np.array(actual_diag)
test_predictions = np.array(test_predictions)


In [ ]:
actual_diag.size

1026

In [ ]:
scores=loaded_model.evaluate(x,actual_diag)
scores

33/33 [==============================] - 0s 3ms/step - loss: 0.3635 - accuracy: 0.8255


[0.3634575307369232, 0.8255360722541809]

In [ ]:
accuracy=accuracy_score(actual_diag,binary_predictions)

In [ ]:
accuracy

0.8255360623781677